In [ ]:
import glob
import os

import numpy
import pyknon.genmidi
import pyknon.music
import pytuning.tuning_tables
import sklearn.preprocessing

In [ ]:
REFERENCE_NOTE = 69 # 440 Hz, nice

In [ ]:
scale = pytuning.create_edo_scale(22)

table = pytuning.tuning_tables.create_csound_tuning(scale, REFERENCE_NOTE)

with open("data/22-EDO.table", "w") as fp:
    fp.write(table)

In [ ]:
AMINO_ACIDS = {
    "A": ( 0,  89.094,  1.8),
    "C": ( 1, 121.154,  2.5),
    "D": ( 2, 133.104, -3.5),
    "E": ( 3, 147.131, -3.5),
    "F": ( 4, 165.192,  2.8),
    "G": ( 5,  75.067, -0.4),
    "H": ( 6, 155.156, -3.2),
    "I": ( 7, 131.175,  4.5),
    "K": ( 8, 146.189, -3.9),
    "L": ( 9, 131.175,  3.8),
    "M": (10, 149.208,  1.9),
    "N": (11, 132.119, -3.5),
    "P": (12, 115.132, -1.6),
    "Q": (13, 146.146, -3.5),
    "R": (14, 174.203, -4.5),
    "S": (15, 105.093, -0.8),
    "T": (16, 119.119, -0.7),
    "U": (17, 168.064,  0.0),
    "V": (18, 117.148,  4.2),
    "W": (19, 204.228, -0.9),
    "Y": (20, 181.191, -1.3)
}

In [ ]:
masses = numpy.array([x for _, x, _ in AMINO_ACIDS.values()])[:, numpy.newaxis]
scales = numpy.array([x for _, _, x in AMINO_ACIDS.values()])[:, numpy.newaxis]

masses = sklearn.preprocessing.normalize(masses, axis=0)
scales = sklearn.preprocessing.normalize(scales, axis=0)

scales = numpy.absolute(scales)
scales = sklearn.preprocessing.minmax_scale(scales, feature_range=(50, 100), axis=0)
scales = numpy.int16(scales)

masses = masses.ravel()
scales = scales.ravel()

features = []

for x, (y, z) in enumerate(zip(masses, scales)):
    features += [(x, y, z)]

In [ ]:
pathnames = glob.glob("data/fasta/*.fasta")

for pathname in pathnames:
    name = os.path.splitext(os.path.split(pathname)[-1])[0].split("_")[-1]

    with open(pathname) as fp:
        sequence = fp.read()

    sequence = sequence.splitlines()[-1].strip()
    
    notes = []

    records = [dict(zip(AMINO_ACIDS.keys(), features))[amino_acid] for amino_acid in sequence]

    for index, dur, volume in records:
        notes += [pyknon.music.Note(index, dur=dur, volume=volume)]

    midi = pyknon.genmidi.Midi()

    midi.seq_notes(pyknon.music.NoteSeq(notes))

    midi.write(f"data/midi/{name.lower()}.midi")